<a href="https://colab.research.google.com/github/Meguazy/HACK_4DS/blob/main/notebooks_luca/HACK4DS_Day2_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Meguazy/HACK_4DS.git

Cloning into 'HACK_4DS'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 106 (delta 44), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (106/106), 11.92 MiB | 4.75 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Updating files: 100% (15/15), done.


In [2]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [3]:
%cd /content/HACK_4DS

/content/HACK_4DS


# Initial

In [4]:
!pip install lightgbm imbalanced-learn

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.pyplot as plt

In [6]:
df_test = pd.read_csv("data2/test_set.csv", sep=";", quotechar="\"", decimal=",")

In [7]:
df_train = pd.read_csv("data2/train_set.csv", sep=";", quotechar="\"", decimal=",")

# Preprocessing

In [8]:
external_score_ver03_dictionary = {
    "MISSING":None,
    "A":14,
    "B":13,
    "C":12,
    "D":11,
    "E":10,
    "F":9,
    "G":8,
    "H":7,
    "I":6,
    "L":5,
    "M":4,
    "N":3,
    "O":2,
    "P":1
}

df_train = df_train.replace({"external_score_ver03": external_score_ver03_dictionary})
df_test = df_test.replace({"external_score_ver03": external_score_ver03_dictionary})

In [9]:
# Concatenate df_train and df_test
df_combined = pd.concat([df_train, df_test], axis=0)

# List of columns to encode
columns_to_encode = ['province', 'juridical_form', 'industry_sector', 'region', 'geo_area', 'decision_date']

# Instantiate LabelEncoder
label_encoders = {}

# Encode each column iteratively
for column in columns_to_encode:
    # Instantiate LabelEncoder for the current column
    label_encoders[column] = LabelEncoder()

    # Fit label encoder on combined data
    label_encoders[column].fit(df_combined[column])

    # Transform both training and testing data
    df_train[column] = label_encoders[column].transform(df_train[column])
    df_test[column] = label_encoders[column].transform(df_test[column])

In [10]:
class_columns = ['decision_date', 'company_ID', 'external_score_ver01',
       'external_score_ver02', 'late_payment_score',
       'external_score_late_payment_integrated', 'external_score_moderate',
       'external_score_adverse', 'external_score_ver03', 'age', 'province',
       'juridical_form', 'industry_sector', 'gross_margin_ratio',
       'core_income_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'cr_available', 'region', 'geo_area', 'last_statement_age',
       'overrun_freq_a_revoca_autoliquidanti',
       'avg_tension_a_revoca_autoliquidanti',
       'std_tension_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'last_tension_a_revoca_autoliquidanti',
       'avg_rel_used_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti',
       'max_rel_used_a_revoca_autoliquidanti',
       'last_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'avg_rel_used_a_scadenza', 'std_rel_used_a_scadenza',
       'max_rel_used_a_scadenza', 'last_rel_used_a_scadenza',
       'avg_count_enti_affidanti', 'std_count_enti_affidanti',
       'max_count_enti_affidanti', 'last_count_enti_affidanti',
       'avg_count_numero_prima_info', 'std_count_numero_prima_info',
       'max_count_numero_prima_info', 'last_count_numero_prima_info']

reg_columns = ['decision_date', 'company_ID', 'external_score_ver01',
       'external_score_ver02', 'external_score_ver03', 'age', 'province',
       'juridical_form', 'industry_sector', 'gross_margin_ratio',
       'core_income_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'cr_available', 'region', 'geo_area', 'last_statement_age',
       'overrun_freq_a_revoca_autoliquidanti',
       'avg_tension_a_revoca_autoliquidanti',
       'std_tension_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'last_tension_a_revoca_autoliquidanti',
       'avg_rel_used_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti',
       'max_rel_used_a_revoca_autoliquidanti',
       'last_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'avg_rel_used_a_scadenza', 'std_rel_used_a_scadenza',
       'max_rel_used_a_scadenza', 'last_rel_used_a_scadenza',
       'avg_count_enti_affidanti', 'std_count_enti_affidanti',
       'max_count_enti_affidanti', 'last_count_enti_affidanti',
       'avg_count_numero_prima_info', 'std_count_numero_prima_info',
       'max_count_numero_prima_info', 'last_count_numero_prima_info']

In [11]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train[["days_to_default", "target"]], test_size=0.2, random_state=42)

# Classificator

In [12]:
from sklearn.metrics import f1_score
import lightgbm as lgb
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

train_data = lgb.Dataset(X_train[class_columns], label=y_train.target)


# Setting up parameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05,
    'random_state': 42,
    'is_unbalance': True
}

# Training the model
num_round = 3100
classificator = lgb.train(params, train_data, num_round)

# Making predictions
y_pred_class = classificator.predict(X_test[class_columns], num_iteration=classificator.best_iteration)

# Converting probabilities to binary predictions
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred_class]

# Calculating F1 score
f1 = f1_score(y_test.target, y_pred_binary)
print("F1 score: ", f1)

[LightGBM] [Info] Number of positive: 5545, number of negative: 20080
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.210510 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6166
[LightGBM] [Info] Number of data points in the train set: 25625, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.216390 -> initscore=-1.286828
[LightGBM] [Info] Start training from score -1.286828
F1 score:  0.5978768577494692


# Regressor

In [13]:
# Splitting the data into training and testing sets
# X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(df_train, df_train.days_to_default, test_size=0.2, random_state=42)

In [14]:
# Scaling the features
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# X_train_scaled = scaler.fit_transform(X_train.loc[X_train["target"] == 1][reg_columns])
# X_test_scaled = scaler.transform(X_test.loc[X_test["target"] == 1][reg_columns])
# df_test_scaled = scaler.transform(df_test[reg_columns])
X_train_scaled = X_train[reg_columns]
X_test_scaled = X_test[reg_columns]
df_test_scaled = df_test[reg_columns]

In [15]:
from sklearn.metrics import mean_absolute_error as mae
from lightgbm import LGBMRegressor

regressor = LGBMRegressor(
    objective='regression',
    boosting_type='gbdt',
    num_leaves=51,
    learning_rate=0.18,
    n_estimators=200,
    feature_fraction=0.88,
    bagging_fraction=0.8,
    num_iterations=6000,
    bagging_freq=5,
    verbose=-1
)

regressor.fit(X_train_scaled, y_train.days_to_default)

# Make predictions on the training and validation data.
y_train_pred_reg = regressor.predict(X_train_scaled)
y_pred_reg = regressor.predict(X_test_scaled)

# Calculate and print the Root Mean Absolute Error (MAE) for training and validation predictions.
print("Training MAE: ", mae(y_train.days_to_default, y_train_pred_reg))
print("Validation MAE: ", mae(y_test.days_to_default, y_pred_reg))

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training MAE:  0.2361893075122995
Validation MAE:  244.61924728545512


# Local Validation

In [20]:
df_local_val = X_test.copy()

# Making predictions
y_class_local = classificator.predict(df_local_val[class_columns], num_iteration=classificator.best_iteration)

# Converting probabilities to binary predictions
y_class_local_binary = [1 if pred > 0.5 else 0 for pred in y_class_local]

df_local_val['default_prediction'] = y_class_local_binary

In [21]:
df_local_val['reg_prediction'] = df_local_val.apply(lambda row: regressor.predict([row[reg_columns].values])[0] if row['default_prediction'] == 1 else 1498, axis=1)

In [22]:
df_local_val

,application_ID,decision_date,company_ID,external_score_ver01,external_score_ver02,late_payment_score,external_score_late_payment_integrated,external_score_moderate,external_score_adverse,external_score_ver03,...,max_count_enti_affidanti,last_count_enti_affidanti,avg_count_numero_prima_info,std_count_numero_prima_info,max_count_numero_prima_info,last_count_numero_prima_info,days_to_default,target,default_prediction,reg_prediction
1786,a1Q7R00000eiXseUAE,682,7.210594e+09,8,2,7.0,8.0,8.0,8.0,12.0,...,2,2,3.166667,0.577350,5,5,1498,0,1,1068.544895
24477,a1Q7R00000eiopnUAA,694,9.085873e+09,6,3,NaN,NaN,NaN,NaN,7.0,...,1,1,1.000000,0.000000,1,1,1498,0,0,1498.000000
21064,a1Q7R00000eiRoWUAU,645,1.708406e+09,8,3,NaN,NaN,NaN,NaN,7.0,...,1,1,1.000000,0.000000,1,1,1498,0,0,1498.000000
24841,a1Q7R00000eiOljUAE,631,3.153185e+09,6,2,NaN,NaN,NaN,NaN,7.0,...,1,1,1.833333,1.029857,3,3,112,1,0,1498.000000
1655,a1Q7R00000eiMBFUA2,610,2.429153e+09,5,1,12.0,3.0,4.0,4.0,10.0,...,2,2,1.083333,0.288675,2,2,1498,0,0,1498.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18547,a1Q7R00000g5y4LUAQ,819,7.840150e+09,9,2,NaN,NaN,NaN,NaN,NaN,...,11,11,0.000000,0.000000,0,0,508,1,0,1498.000000
26968,a1Q7R00000a3C1wUAE,557,3.244555e+08,4,1,NaN,NaN,NaN,NaN,6.0,...,13,13,1.000000,0.000000,1,1,1498,0,0,1498.000000
3753,a1Q7R00000c7PWMUA2,868,1.287521e+09,7,2,NaN,NaN,NaN,NaN,9.0,...,5,5,2.000000,0.000000,2,2,1498,0,0,1498.000000
22753,a1Q7R00000ZWVLMUA5,443,5.390194e+09,4,1,NaN,NaN,NaN,NaN,4.0,...,3,3,2.916667,0.514929,4,4,1498,0,0,1498.000000


In [23]:
print("Validation Local MAE: ", mae(df_local_val.days_to_default, df_local_val.reg_prediction))

Validation Local MAE:  157.9622209185935


# Submit solution

In [ ]:
df_test_val = df_test.copy()

# Making predictions
y_final = classificator.predict(df_test_val[class_columns], num_iteration=classificator.best_iteration)

# Converting probabilities to binary predictions
y_final_binary = [1 if pred > 0.5 else 0 for pred in y_final]

df_test_val['default_prediction'] = y_final_binary

df_test_val['reg_prediction'] = df_test_val.apply(lambda row: regressor.predict([row[reg_columns].values])[0] if row['default_prediction'] == 1 else 1498, axis=1)

In [ ]:
submission = pd.DataFrame(df_test_val['reg_prediction'])

In [ ]:
submission.to_csv("file.csv", index=False, header=False)